In [1]:
import wandb
import os 
import numpy as np
import pandas as pd
import firebase_admin
from firebase_admin import credentials, firestore, storage
from tqdm import tqdm 

/tmp/ipykernel_13203/1617223376.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Initialize Firebase Admin SDK
cred = credentials.Certificate('disaster-master.json')  # Replace with your Firebase Admin SDK key
firebase_admin.initialize_app(cred, {
    'storageBucket': 'disaster-master-59d6f.appspot.com'  # Replace with your storage bucket URL
})

db = firestore.client()
bucket = storage.bucket()

In [3]:
"""
provide a function for this 
Check firestore db 

Check the users table -> check gameEnded true -> log with prefix `All` -> store all users. `CorrectlyAnswered`, `GatherAdditionalDataRequested`, `WronglyAnswered` , `score` in a list. Take the average and log to wandb
Check the users table -> Check gameEnded true -> log with prefix `victim` -> check users.experience.victim true -> store all users. `CorrectlyAnswered`, `GatherAdditionalDataRequested`, `WronglyAnswered` , `score` in a list. Take the average and log to wandb
Check the users table -> Check gameEnded true -> log with prefix `volunteer` -> check users.experience.volunteer true -> store all users. `CorrectlyAnswered`, `GatherAdditionalDataRequested`, `WronglyAnswered` , `score` in a list. Take the average and log to wandb
Check the users table -> Check gameEnded true -> log with prefix `decision_maker` -> check users.experience.decision_maker true -> store all users. `CorrectlyAnswered`, `GatherAdditionalDataRequested`, `WronglyAnswered` , `score` in a list. Take the average and log to wandb
"""



'\nprovide a function for this \nCheck firestore db \n\nCheck the users table -> check gameEnded true -> log with prefix `All` -> store all users. `CorrectlyAnswered`, `GatherAdditionalDataRequested`, `WronglyAnswered` , `score` in a list. Take the average and log to wandb\nCheck the users table -> Check gameEnded true -> log with prefix `victim` -> check users.experience.victim true -> store all users. `CorrectlyAnswered`, `GatherAdditionalDataRequested`, `WronglyAnswered` , `score` in a list. Take the average and log to wandb\nCheck the users table -> Check gameEnded true -> log with prefix `volunteer` -> check users.experience.volunteer true -> store all users. `CorrectlyAnswered`, `GatherAdditionalDataRequested`, `WronglyAnswered` , `score` in a list. Take the average and log to wandb\nCheck the users table -> Check gameEnded true -> log with prefix `decision_maker` -> check users.experience.decision_maker true -> store all users. `CorrectlyAnswered`, `GatherAdditionalDataRequested

In [4]:
# Initialize WandB
wandb.init(project="human_study")

def log_to_wandb(prefix, data):
    def calculate_mean_std(values):
        return np.mean(values), np.std(values)

    # Prepare data arrays
    correctly_answered = [d['CorrectlyAnswered'] for d in data]
    gather_additional_data = [d['GatherAdditionalDataRequested'] for d in data]
    wrongly_answered = [d['WronglyAnswered'] for d in data]
    scores = [d['score'] for d in data]
    trees_done = [d['trees'] for d in data]

    # Calculate mean and std for each metric
    mean_correctly_answered, std_correctly_answered = calculate_mean_std(correctly_answered)
    mean_gather_additional_data, std_gather_additional_data = calculate_mean_std(gather_additional_data)
    mean_wrongly_answered, std_wrongly_answered = calculate_mean_std(wrongly_answered)
    mean_score, std_score = calculate_mean_std(scores)
    total_trees_done = sum(trees_done)
    mean_trees_done, std_trees_done = calculate_mean_std(trees_done)

    # Log mean and std to WandB
    wandb.log({f"{prefix}/CorrectlyAnswered_mean": mean_correctly_answered, f"{prefix}/CorrectlyAnswered_std": std_correctly_answered})
    wandb.log({f"{prefix}/GatherAdditionalDataRequested_mean": mean_gather_additional_data, f"{prefix}/GatherAdditionalDataRequested_std": std_gather_additional_data})
    wandb.log({f"{prefix}/WronglyAnswered_mean": mean_wrongly_answered, f"{prefix}/WronglyAnswered_std": std_wrongly_answered})
    wandb.log({f"{prefix}/score_mean": mean_score, f"{prefix}/score_std": std_score})
    wandb.log({f"{prefix}/total_trees_done": total_trees_done})
    wandb.log({f"{prefix}/avg_trees_done_mean": mean_trees_done, f"{prefix}/avg_trees_done_std": std_trees_done})

    # Log individual data points
    step = 0
    for i, d in enumerate(data):
        wandb.log({f"{prefix}/CorrectlyAnswered": d["CorrectlyAnswered"], "step": step})
        wandb.log({f"{prefix}/GatherAdditionalDataRequested": d["GatherAdditionalDataRequested"], "step": step})
        wandb.log({f"{prefix}/WronglyAnswered": d["WronglyAnswered"], "step": step})
        wandb.log({f"{prefix}/score": d["score"], "step": step})
        wandb.log({f"{prefix}/trees_done": d["trees"], "step": step})
        step += 1

def normalize_nationality(nationality):
    nationality = nationality.strip().lower()
    
    nationality_mapping = {
        "india": "indian",
        "indian": "indian",
        "indian ": "indian",
        "indian  ": "indian",
        "indian": "indian",
        "koshy george": "indian",
        "indian": "indian",
        "us": "american",
        "usa": "american",
        "u.s.": "american",
        "u.s.a": "american",
        "british": "british",
        "british ": "british",
    }

    return nationality_mapping.get(nationality, nationality)  # Default to original if not in mapping

def process_users_data():
    users_ref = db.collection('users')
    users = users_ref.where('gameEnded', '==', True).stream()

    all_data = []
    victim_data = []
    volunteer_data = []
    decision_maker_data = []

    nationalities = set()  # Initialize an empty set to store unique nationalities
    nationality_stats = {}  # Dictionary to store counts for each nationality

    for user in users:
        user_data = user.to_dict()
        
        nationality = user_data.get("nationality")
        if nationality:
            normalized_nationality = normalize_nationality(nationality)
            nationalities.add(normalized_nationality)
            if normalized_nationality not in nationality_stats:
                nationality_stats[normalized_nationality] = {
                    "victim": 0,
                    "volunteer": 0,
                    "decision_maker": 0
                }

        user_stats = {
            "CorrectlyAnswered": user_data.get("CorrectlyAnswered", 0),
            "GatherAdditionalDataRequested": user_data.get("GatherAdditionalDataRequested", 0),
            "WronglyAnswered": user_data.get("WronglyAnswered", 0),
            "score": user_data.get("score", 0),
            "trees": user_data.get("current_tree_level", 0)
        }
        all_data.append(user_stats)

        if user_data.get("experience", {}).get("victim"):
            victim_data.append(user_stats)
            if nationality:
                nationality_stats[normalized_nationality]["victim"] += 1
        
        if user_data.get("experience", {}).get("volunteer"):
            volunteer_data.append(user_stats)
            if nationality:
                nationality_stats[normalized_nationality]["volunteer"] += 1
        
        if user_data.get("experience", {}).get("decision_maker"):
            decision_maker_data.append(user_stats)
            if nationality:
                nationality_stats[normalized_nationality]["decision_maker"] += 1

    # Print the unique set of nationalities
    print("Unique Nationalities:", nationalities)

    # Log nationality stats to WandB
    for nationality, counts in nationality_stats.items():
        wandb.log({
            "nationality_counts/victims": {nationality: counts["victim"]},
            "nationality_counts/volunteers": {nationality: counts["volunteer"]},
            "nationality_counts/decision_makers": {nationality: counts["decision_maker"]}
        })

    if all_data:
        log_to_wandb("All", all_data)
    
    if victim_data:
        wandb.log({"victim/number_of_victims": len(victim_data)})
        log_to_wandb("victim", victim_data)
    
    if volunteer_data:
        wandb.log({"volunteer/number_of_volunteer": len(volunteer_data)})
        log_to_wandb("volunteer", volunteer_data)
    
    if decision_maker_data:
        wandb.log({"decision_maker/number_of_decision_maker": len(decision_maker_data)})
        log_to_wandb("decision_maker", decision_maker_data)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: juliangeralddcruz. Use `wandb login --relogin` to force relogin


In [5]:
process_users_data()

/home/julian/miniconda3/envs/pytorch_env/lib/python3.10/site-packages/google/cloud/firestore_v1/base_collection.py:302: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)


Unique Nationalities: {'taiwan', 'italian', 'indian', 'american', 'british', 'testing', 'japan', 'canadian'}


In [6]:
wandb.finish()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


All/CorrectlyAnswered,▆▇▅▆▄▇▄▇▅▆▇▆█▇▇▅▅▆▆▅▅▄▆▅▂▃▁▄▄▆▃▇▅▅▆▃▆▅▅▄
All/CorrectlyAnswered_mean,▁
All/CorrectlyAnswered_std,▁
All/GatherAdditionalDataRequested,▆▁▁▅▃▄▁▁▁▇▆▁▁▂█▅▁▁▁▅▄▄▆▁▄▃▁▂▂▂▅▄▂▂▄▁▂▄▁▃
All/GatherAdditionalDataRequested_mean,▁
All/GatherAdditionalDataRequested_std,▁
All/WronglyAnswered,▃▂▄▃▃▂▅▂▄▃▂▃▁▂▂▄▄▃▃▄▄▅▃▄▅▅█▅▅▃▄▂▄▄▃▆▃▄▃▄
All/WronglyAnswered_mean,▁
All/WronglyAnswered_std,▁
All/avg_trees_done_mean,▁
All/avg_trees_done_std,▁
